# SDSU Data

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df = pd.read_excel(io='SDSU_mech_eng_classes.xlsx', header=1)

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Department      96 non-null     object 
 1   Subject         96 non-null     object 
 2   Course #        96 non-null     int64  
 3   Suffix          13 non-null     object 
 4   Catalog #       0 non-null      float64
 5   Title           96 non-null     object 
 6   Section         96 non-null     int64  
 7   Schedule #      96 non-null     object 
 8   Units           96 non-null     int64  
 9   Session         96 non-null     object 
 10  Seats           96 non-null     int64  
 11  Available       96 non-null     int64  
 12  Total Enrolled  96 non-null     int64  
 13  Blocked         0 non-null      float64
 14  Footnotes       89 non-null     object 
 15  SDSU Enrolled   0 non-null      float64
 16  EE Enrolled     0 non-null      float64
 17  SDSU Audit      0 non-null      float

In [4]:
df = df.drop(['Catalog #', 'Blocked', 'SDSU Enrolled', 'EE Enrolled', 'SDSU Audit', 'EE Audit'], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Department      96 non-null     object 
 1   Subject         96 non-null     object 
 2   Course #        96 non-null     int64  
 3   Suffix          13 non-null     object 
 4   Title           96 non-null     object 
 5   Section         96 non-null     int64  
 6   Schedule #      96 non-null     object 
 7   Units           96 non-null     int64  
 8   Session         96 non-null     object 
 9   Seats           96 non-null     int64  
 10  Available       96 non-null     int64  
 11  Total Enrolled  96 non-null     int64  
 12  Footnotes       89 non-null     object 
 13  Meeting Type    96 non-null     object 
 14  Building        84 non-null     object 
 15  Room            84 non-null     float64
 16  Start Time      84 non-null     float64
 17  End Time        84 non-null     float

Convert to special Int64 type because there are null values in the columns. (pandas nullable integrer data type)

In [5]:
# df['Room'] = df['Room'].astype('Int64')
# df['Start Time'] = df['Start Time'].astype('Int64')
# df['End Time'] = df['End Time'].astype('Int64')
# print(df.info())

In [6]:
# df['Course #'] = df['Course #'].astype('int32')

In [7]:
# print(df.iloc[0])
# print(df.iloc[0]['Subject'])
# print(df.iloc[0][1])
# print(tuple(df.iloc[0].to_list()))

In [8]:
sqlite3.register_adapter(np.int64, lambda val: int(val))
# sqlite3.register_adapter(np.int32, lambda val: int(val))

In [9]:
con = sqlite3.connect("SDSU_mech_eng_classes.sqlite")

In [10]:
cur = con.cursor()

In [11]:
cur.execute('DROP TABLE IF EXISTS Classes')
cur.execute('''CREATE TABLE Classes 
            (Department TEXT, Subject TEXT, CourseNum INTEGER, Suffix TEXT, Title TEXT, Section INTEGER, ScheduleNum TEXT, 
            Units INTEGER, Session TEXT, Seats INTEGER, Available INTEGER, TotalEnrolled INTEGER, Footnotes TEXT, 
            MeetingType TEXT, Building TEXT, Room INTEGER, StartTime INTEGER, EndTime INTEGER, Days TEXT, Instructor TEXT)''')


In [12]:
for i in range(len(df)):
    cur.execute('''INSERT INTO Classes
                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', tuple(df.iloc[i].to_list()))
       
con.commit()

Counting how many classes are fully filled up with 0 seats remaining, and how many classes are still available.

In [13]:
cur.execute('SELECT COUNT(*) FROM Classes WHERE Available <= 0')
x = cur.fetchall()
print('There are ' + str(x[0][0]) + ' classes that are fully filled.')
cur.execute('SELECT COUNT(*) FROM Classes WHERE Available > 0')
x = cur.fetchall()
print('There are ' + str(x[0][0]) + ' classes that are still available.')

There are 27 classes that are fully filled.
There are 69 classes that are still available.


Lets remove all of the classes that are fully filled since we can't enroll in them.

In [14]:
cur.execute('DELETE FROM Classes WHERE Available <= 0')
con.commit()

Maybe you want your early class (9AM or earlier) to be at building E which is closest to you.

In [21]:
cur.execute('SELECT Title, StartTime FROM Classes WHERE StartTime <= 900 AND Building = ?', ('E',))
cur.fetchall()

[('COMPUTER-AIDED DESIGN', 800),
 ('COMPUTER-AIDED DESIGN', 900),
 ('COMPUTER-AIDED DESIGN', 800),
 ('COMPUTER-AIDED DESIGN', 800),
 ('COMPUTER-AIDED DESIGN', 900),
 ('MECHATRONICS LABORATORY', 800),
 ('ENRGY& THRM SYS ANAL/DSGN', 800),
 ('FUNDAMENTALS OF NANOBIO', 800)]

Or if you want to see all the classes that your favorite teacher C. SAGHAFI is teaching that aren't on Fridays.

In [22]:
cur.execute('SELECT Title, Instructor, Days FROM Classes WHERE Instructor = ? AND Days != ?', ('C. SAGHAFI','F'))
cur.fetchall()

[('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'M'),
 ('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'M'),
 ('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'W'),
 ('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'W'),
 ('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'M'),
 ('COMPUTER-AIDED DESIGN', 'C. SAGHAFI', 'M')]

In [15]:
# con.close()